# Random Forest Model with natural data

## 1 Feature engineering

### 1.1 Importing libraries, transformers and path setting  

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

### 1.2  Data loading

In [3]:
from app_local.module import DataEnsembler, GestureTransformer, DataResampler

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.combined_data_files_df


,filename_features,filename_labels
0,features_left_c_02_120.csv,labels_left_c_02.csv
1,features_left_l_01_120.csv,labels_left_l_01.csv
2,features_move_p_01_120.csv,labels_move_p_01.csv
3,features_move_c_01_120.csv,labels_move_c_01.csv
4,features_land_l_03_120.csv,labels_land_l_03.csv
5,features_move_l_02_120.csv,labels_move_l_02.csv
6,features_land_c_01_120.csv,labels_land_c_01.csv
7,features_land_l_02_120.csv,labels_land_l_02.csv
8,features_left_c_03_120.csv,labels_left_c_03.csv
9,features_right_c_02_120.csv,labels_right_c_02.csv


In [4]:
de.load_data()
de.pattern

'(?P<filename>(?P<filetype>[a-z]*)_(?P<movement>[a-z]*)_(?P<person>[a-z]*)_(?P<filenum>\\d*)(_(?P<frame_length>\\d*))?\\.csv)'

### 1.3 Data rescaling to actual video length

In [ ]:
#de.rescale_data_frames(time_of_first_frame='avg',verbose=True)

### 1. 4 Generating training data

In [5]:
de.assemble_data(tolerance_range= 2)
de.display_information()


i: 0 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 114
i: 1 	shape X: (541, 18, 16) 	shape y: (541,) 	count: 92
i: 2 	shape X: (590, 18, 16) 	shape y: (590,) 	count: 171
i: 3 	shape X: (616, 18, 16) 	shape y: (616,) 	count: 136
i: 4 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 126
i: 5 	shape X: (470, 18, 16) 	shape y: (470,) 	count: 187
i: 6 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 127
i: 7 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 87
i: 8 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 115
i: 9 	shape X: (923, 18, 16) 	shape y: (923,) 	count: 214
i: 10 	shape X: (545, 18, 16) 	shape y: (545,) 	count: 91
i: 11 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 118
i: 12 	shape X: (621, 18, 16) 	shape y: (621,) 	count: 269
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 127
i: 14 	shape X: (505, 18, 16) 	shape y: (505,) 	count: 111
i: 15 	shape X: (535, 18, 16) 	shape y: (535,) 	count: 121
i: 16 	shape X: (514, 18, 16) 	shape y: (514,) 	count: 120
i: 17 	sha

### 1.6 Data normalization

In [6]:
X = de.X.copy()
y = de.y.copy().astype("int32")
gt = GestureTransformer(byrow=True, feature_names= list(de.feature_names))
X = gt.transform(X, verbose = True)

---------------------------------------------------------
Performing row-based transformations:
X-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19959, 18)
Y-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (19959, 18)
Scaling based on Hip-Shoulder distance:
	Row-based distance is calculated for each sample and each timestep
	Shape: (19959, 18)
-----------------------------------------------------
X-Shift:
            min   mean  median    max   std
timestep                                   
0         196.8  294.4   282.6  511.9  37.5
1         196.8  294.4   282.6  511.9  37.5
2         196.8  294.3   282.5  511.9  37.6
3         196.8  294.3   282.4  511.9  37.6
4         196.8  294.2   282.4  511.9  37.6
5         196.8  294.2   282.3  511.9  37.6
6         196.8  294.2   282.3  511.9  37.6
7         196.8  294.1   282.2  511.9  37.6
8         196.8  294.1   282.2  511.9  37.6
9         1

### 1.5 Imbalance data treatment

In [7]:
dr = DataResampler()
dr.fit(de.X, de.y)
#dr.restrict_zero_labels(n=500, criterion=["variances_lowest"])
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,15273,76.52,15273,76.52
1,701,3.51,701,3.51
2,763,3.82,763,3.82
3,991,4.97,991,4.97
4,882,4.42,882,4.42
5,660,3.31,660,3.31
6,689,3.45,689,3.45


In [8]:
dr.restrict_zero_labels(n=600,criterion="variances_lowest")
dr.display_information()

-------------------------------------
Current number of zero-labels: 15273
New number of zero-labels (selected with criterion="variances_lowest"): 600


,abs_orig,pct_orig,abs_current,pct_current
0,15273,76.52,600,11.35
1,701,3.51,701,13.26
2,763,3.82,763,14.43
3,991,4.97,991,18.75
4,882,4.42,882,16.69
5,660,3.31,660,12.49
6,689,3.45,689,13.03


## 2 Model Training

### 2. 1 OneHot-Encoding of target vector

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
ohe = OneHotEncoder(sparse = False)
print(type(ohe))
target = ohe.fit_transform(y.reshape(-1,1))
print("Before:", y.shape)
print("After:", target.shape)
target[0:5,:]

<class 'sklearn.preprocessing.data.OneHotEncoder'>
Before: (19959,)
After: (19959, 7)


array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.]])

### 2. 2 Train-Test-Split

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=10)
print("Training Data:", x_train.shape, y_train.shape)
print("Test Data:", x_test.shape, y_test.shape)

Training Data: (15967, 18, 16) (15967, 7)
Test Data: (3992, 18, 16) (3992, 7)


### 2. 3 Model architecture

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### Train a simple Random Forest

In [13]:
model = RandomForestClassifier(
    n_estimators = 100, # 100
    criterion = "gini", # {gini, entropy}
    max_depth = None , # None
    n_jobs = -1,
    verbose = 1
)

model.fit(x_train.reshape(x_train.shape[0],-1),y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.4s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

#### Training accuracy is already 100%

In [14]:
preds = model.predict(x_train.reshape(x_train.shape[0],-1))
(preds == y_train).all(axis = 1).sum()/y_train.shape[0]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


1.0

#### Test accuracy is 95%

In [15]:
preds = model.predict(x_test.reshape(x_test.shape[0],-1))
(preds == y_test).all(axis = 1).sum()/y_test.shape[0]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


0.9486472945891784

### 2. 4 Gridsearch with Crossvalidation

In [16]:
# Number of trees in random forest
n_estimators = [100, 500, 1000]
# Number of features to consider at every split
max_features = [5, 10, 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 4)]
max_depth.append(None)

# Create the random grid
random_grid = {'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__max_depth': max_depth,
              }

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              }

print(random_grid)

{'n_estimators': [100, 500, 1000], 'max_features': [5, 10, 'sqrt'], 'max_depth': [5, 40, 75, 110, None]}


In [17]:
model.get_params().keys()

dict_keys(['bootstrap', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

#### Training takes quite a long time - do not needlessly repeat it

In [18]:
params = random_grid
scoring = "accuracy"
clf = GridSearchCV(model, params, n_jobs = -1, verbose = 10, scoring = scoring)
clf.fit(x_train.reshape(x_train.shape[0],-1),y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits
[CV] max_depth=5, max_features=5, n_estimators=100 ...................
[CV] max_depth=5, max_features=5, n_estimators=100 ...................
[CV] max_depth=5, max_features=5, n_estimators=100 ...................
[CV] max_depth=5, max_features=5, n_estimators=500 ...................
[CV] max_depth=5, max_features=5, n_estimators=500 ...................
[CV] max_depth=5, max_features=5, n_estimators=500 ...................
[CV] max_depth=5, max_features=5, n_estimators=1000 ..................
[CV] max_depth=5, max_features=5, n_estimators=1000 ..................


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished


[CV]  max_depth=5, max_features=5, n_estimators=100, score=0.8395641555513808, total=   2.9s
[CV] max_depth=5, max_features=5, n_estimators=1000 ..................


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished


[CV]  max_depth=5, max_features=5, n_estimators=100, score=0.8361518226230741, total=   3.1s


...

[CV]  max_depth=None, max_features=sqrt, n_estimators=1000, score=0.943630214205186, total= 5.4min


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  3.7min finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    1.5s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    2.1s finished


[CV]  max_depth=None, max_features=sqrt, n_estimators=1000, score=0.9498308906426156, total= 3.9min


[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 45.1min finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   53.1s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 500, 1000], 'max_features': [5, 10, 'sqrt'], 'max_depth': [5, 40, 75, 110, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=10)

In [19]:
clf.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_max_features', 'param_n_estimators', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'mean_train_score', 'std_train_score'])

In [ ]:
clf.cv_results_["mean_test_score"]

#### Determine performance of best Gridsearch-Model on the Holdout-Set

In [20]:
preds = clf.best_estimator_.predict(x_test.reshape(x_test.shape[0],-1))
(preds == y_test).all(axis = 1).sum()/y_test.shape[0]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.6s finished


0.9506513026052105

In [21]:
from sklearn.externals import joblib
joblib.dump(clf.best_estimator_,'models/model_gesture_rf_nip120_sa600_variances_lowest.pkl')

['models/model_gesture_rf_nip120_sa600_variances_lowest.pkl']

In [24]:
from sklearn.externals import joblib
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

loaded_model=joblib.load('models/model_gesture_rf_nip120_sa600_variances_lowest.pkl')
loaded_model

0 /Users/lsafari/drone_steering


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=75, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [28]:
type(loaded_model)

sklearn.ensemble.forest.RandomForestClassifier

In [ ]:
loaded_model.verbose = 0

In [26]:
x12907 = joblib.load('models/x12907.pkl')

In [29]:
loaded_model.predict(x12907)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished


array([[1., 0., 0., 0., 0., 0., 0.]])

In [30]:
loaded_model.predict(X.reshape(X.shape[0],-1))

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.3s finished


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])